### Ingestión de carpeta "production_country"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

#### Paso 1 - Leer archvivos JSON usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [0]:
productions_countries_schema = StructType(fields= [
    StructField("movieId", IntegerType(), True),
    StructField("countryId", IntegerType(), True)
])                       

In [0]:
productions_countries_df = spark.read \
        .schema(productions_countries_schema) \
        .option("multiline", True) \
        .json(f"{bronze_folder_path}/{v_file_date}/production_country") 

#### Pase 2 - Renombrar las columnas y añadir nuevas columnas
1. "movieId" renombrar a "movie_id"
2. "countryId" renombrar a "country_id"
33. Agregar la columnas "ingestion_date" y "environment"

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
productions_countries_final_df = add_ingestion_date(productions_countries_df) \
    .withColumnsRenamed({"movieId" : "movie_id",
                         "countryId" : "country_id"}) \
    .withColumn("environment", lit(v_environment)) \
    .withColumn("file_date", lit(v_file_date))


#### Paso 3 - Escribir datos en el DataLake en formato "Parquet"

In [0]:
#overwrite_partition(productions_countries_final_df, "movie_silver","productions_countries", "file_date")

In [0]:
merge_condition = "tgt.movie_id = src.movie_id AND tgt.country_id = src.country_id AND tgt.file_date = src.file_date"
merge_delta_lake(productions_countries_final_df, "movie_silver","productions_countries", silver_folder_path, merge_condition, "file_date")
#productions_countries_final_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.productions_countries")
#productions_countries_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/productions_countries")

In [0]:
%sql
SELECT file_date, COUNT(1) 
FROM movie_silver.productions_countries
GROUP BY file_date;


In [0]:
%sql
SELECT * FROM movie_silver.productions_countries;

In [0]:
#display(spark.read.parquet(f"{silver_folder_path}/productions_countries"))

In [0]:
dbutils.notebook.exit("Success")